In [ ]:
import numpy as np
import pandas as pd

# Indexing and selecting data

The axis labeling information in pandas objects serves many purposes:

- Identifies data (i.e. provides *metadata*) using known indicators,
  important for analysis, visualization, and interactive console display.  
- Enables automatic and explicit data alignment.  
- Allows intuitive getting and setting of subsets of the data set.  


In this section, we will focus on the final point: namely, how to slice, dice,
and generally get and set subsets of pandas objects. The primary focus will be
on Series and DataFrame as they have received more development attention in
this area.

>**Note**
>
>The Python and NumPy indexing operators `[]` and attribute operator `.`
provide quick and easy access to pandas data structures across a wide range
of use cases. This makes interactive work intuitive, as there’s little new
to learn if you already know how to deal with Python dictionaries and NumPy
arrays. However, since the type of the data to be accessed isn’t known in
advance, directly using standard operators has some optimization limits. For
production code, we recommended that you take advantage of the optimized
pandas data access methods exposed in this chapter.

Whether a copy or a reference is returned for a setting operation, may
depend on the context. This is sometimes called `chained assignment` and
should be avoided. See Returning a View versus Copy.

See the MultiIndex / Advanced Indexing for `MultiIndex` and more advanced indexing documentation.

See the [cookbook](44_cookbook.ipynb#cookbook-selection) for some advanced strategies.


<a id='indexing-choice'></a>

## Different choices for indexing

Object selection has had a number of user-requested additions in order to
support more explicit location based indexing. pandas now supports three types
of multi-axis indexing.

- `.loc` is primarily label based, but may also be used with a boolean array. `.loc` will raise `KeyError` when the items are not found. Allowed inputs are:  
  
  - A single label, e.g. `5` or `'a'` (Note that `5` is interpreted as a
    *label* of the index. This use is **not** an integer position along the
    index.).  
  - A list or array of labels `['a', 'b', 'c']`.  
  - A slice object with labels `'a':'f'` (Note that contrary to usual Python
    slices, **both** the start and the stop are included, when present in the
    index! See Slicing with labels
    and Endpoints are inclusive.)  
  - A boolean array (any `NA` values will be treated as `False`).  
  - A `callable` function with one argument (the calling Series or DataFrame) and
    that returns valid output for indexing (one of the above).  
  
  See more at Selection by Label.  
- `.iloc` is primarily integer position based (from `0` to
  `length-1` of the axis), but may also be used with a boolean
  array.  `.iloc` will raise `IndexError` if a requested
  indexer is out-of-bounds, except *slice* indexers which allow
  out-of-bounds indexing.  (this conforms with Python/NumPy *slice*
  semantics).  Allowed inputs are:  
  
  - An integer e.g. `5`.  
  - A list or array of integers `[4, 3, 0]`.  
  - A slice object with ints `1:7`.  
  - A boolean array (any `NA` values will be treated as `False`).  
  - A `callable` function with one argument (the calling Series or DataFrame) and
    that returns valid output for indexing (one of the above).  
  
  See more at Selection by Position,
  Advanced Indexing and Advanced
  Hierarchical.  
- `.loc`, `.iloc`, and also `[]` indexing can accept a `callable` as indexer. See more at Selection By Callable.  


Getting values from an object with multi-axes selection uses the following
notation (using `.loc` as an example, but the following applies to `.iloc` as
well). Any of the axes accessors may be the null slice `:`. Axes left out of
the specification are assumed to be `:`, e.g. `p.loc['a']` is equivalent to
`p.loc['a', :, :]`.


|Object Type|Indexers|
|:----------------------------|:------------------------------------------------|
|Series|s.loc[indexer]|
|DataFrame|df.loc[row_indexer,column_indexer]|

<a id='indexing-basics'></a>

## Basics

As mentioned when introducing the data structures in the last section, the primary function of indexing with `[]` (a.k.a. `__getitem__`
for those familiar with implementing class behavior in Python) is selecting out
lower-dimensional slices. The following table shows return type values when
indexing pandas objects with `[]`:

|Object Type|Selection|Return Value Type|
|:----------------------------|:----------------------------|:----------------------------------------------------------|
|Series|series[label]|scalar value|
|DataFrame|frame[colname]|Series corresponding to colname|
Here we construct a simple time series data set to use for illustrating the
indexing functionality:

In [ ]:
dates = pd.date_range('1/1/2000', periods=8)
df = pd.DataFrame(np.random.randn(8, 4),
                  index=dates, columns=['A', 'B', 'C', 'D'])
df

>**Note**
>
>None of the indexing functionality is time series specific unless
specifically stated.

Thus, as per above, we have the most basic indexing using `[]`:

In [ ]:
s = df['A']
s[dates[5]]

You can pass a list of columns to `[]` to select columns in that order.
If a column is not contained in the DataFrame, an exception will be
raised. Multiple columns can also be set in this manner:

In [ ]:
df

In [ ]:
df[['B', 'A']] = df[['A', 'B']]
df

You may find this useful for applying a transform (in-place) to a subset of the
columns.

pandas aligns all AXES when setting `Series` and `DataFrame` from `.loc`, and `.iloc`.

This will **not** modify `df` because the column alignment is before value assignment.

In [ ]:
df[['A', 'B']]
df.loc[:, ['B', 'A']] = df[['A', 'B']]
df[['A', 'B']]

The correct way to swap column values is by using raw values:

In [ ]:
df.loc[:, ['B', 'A']] = df[['A', 'B']].to_numpy()
df[['A', 'B']]

## Attribute access


<a id='indexing-columns-multiple'></a>

<a id='indexing-df-cols'></a>

<a id='indexing-attribute-access'></a>
You may access an index on a `Series` or  column on a `DataFrame` directly
as an attribute:

In [ ]:
sa = pd.Series([1, 2, 3], index=list('abc'))
dfa = df.copy()

In [ ]:
sa.b
dfa.A

In [ ]:
sa.a = 5
sa
dfa.A = list(range(len(dfa.index)))  # ok if A already exists
dfa
dfa['A'] = list(range(len(dfa.index)))  # use this form to create a new column
dfa

- You can use this access only if the index element is a valid Python identifier, e.g. `s.1` is not allowed.
  See [here for an explanation of valid identifiers](https://docs.python.org/3/reference/lexical_analysis.html#identifiers).  
- The attribute will not be available if it conflicts with an existing method name, e.g. `s.min` is not allowed, but `s['min']` is possible.  
- Similarly, the attribute will not be available if it conflicts with any of the following list: `index`,
  `major_axis`, `minor_axis`, `items`.  
- In any of these cases, standard indexing will still work, e.g. `s['1']`, `s['min']`, and `s['index']` will
  access the corresponding element or column.  


If you are using the IPython environment, you may also use tab-completion to
see these accessible attributes.

You can also assign a `dict` to a row of a `DataFrame`:

In [ ]:
x = pd.DataFrame({'x': [1, 2, 3], 'y': [3, 4, 5]})
x.iloc[1] = {'x': 9, 'y': 99}
x

You can use attribute access to modify an existing element of a Series or column of a DataFrame, but be careful;
if you try to use attribute access to create a new column, it creates a new attribute rather than a
new column. In 0.21.0 and later, this will raise a `UserWarning`:

```python
In [1]: df = pd.DataFrame({'one': [1., 2., 3.]})
In [2]: df.two = [4, 5, 6]
UserWarning: Pandas doesn't allow Series to be assigned into nonexistent columns - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute_access
In [3]: df
Out[3]:
   one
0  1.0
1  2.0
2  3.0
```


## Slicing ranges

The most robust and consistent way of slicing ranges along arbitrary axes is
described in the Selection by Position section
detailing the `.iloc` method. For now, we explain the semantics of slicing using the `[]` operator.

With Series, the syntax works exactly as with an ndarray, returning a slice of
the values and the corresponding labels:

In [ ]:
s = dfa.A
s[:5]

In [ ]:
s[::2]

In [ ]:
s[::-1]

Note that setting works as well:

In [ ]:
s2 = s.copy()
s2[:5] = 0
s2

With DataFrame, slicing inside of `[]` **slices the rows**. This is provided
largely as a convenience since it is such a common operation.

In [ ]:
df[:3]
df[::-1]


<a id='indexing-label'></a>

## Selection by label

Whether a copy or a reference is returned for a setting operation, may depend on the context.
This is sometimes called `chained assignment` and should be avoided.
See Returning a View versus Copy.

> `.loc` is strict when you present slicers that are not compatible (or convertible) with the index type. For example
using integers in a `DatetimeIndex`. These will raise a `TypeError`.

In [ ]:
dfl = pd.DataFrame(np.random.randn(5, 4),
                   columns=list('ABCD'),
                   index=pd.date_range('20130101', periods=5))
dfl

```python
In [4]: dfl.loc[2:3]
TypeError: cannot do slice indexing on <class 'pandas.tseries.index.DatetimeIndex'> with these indexers [2] of <type 'int'>
```


String likes in slicing *can* be convertible to the type of the index and lead to natural slicing.

In [ ]:
dfl.loc['20130102':'20130104']

Changed in version 1.0.0.

pandas will raise a `KeyError` if indexing with a list with missing labels. See list-like Using loc with
missing keys in a list is Deprecated.

pandas provides a suite of methods in order to have **purely label based indexing**. This is a strict inclusion based protocol.
Every label asked for must be in the index, or a `KeyError` will be raised.
When slicing, both the start bound **AND** the stop bound are *included*, if present in the index.
Integers are valid labels, but they refer to the label **and not the position**.

The `.loc` attribute is the primary access method. The following are valid inputs:

- A single label, e.g. `5` or `'a'` (Note that `5` is interpreted as a *label* of the index. This use is **not** an integer position along the index.).  
- A list or array of labels `['a', 'b', 'c']`.  
- A slice object with labels `'a':'f'` (Note that contrary to usual Python
  slices, **both** the start and the stop are included, when present in the
  index! See Slicing with labels.  
- A boolean array.  
- A `callable`, see Selection By Callable.  

In [ ]:
s1 = pd.Series(np.random.randn(6), index=list('abcdef'))
s1

In [ ]:
s1.loc['c':]

In [ ]:
s1.loc['b']

Note that setting works as well:

In [ ]:
s1.loc['c':] = 0
s1

With a DataFrame:

In [ ]:
df1 = pd.DataFrame(np.random.randn(6, 4),
                   index=list('abcdef'),
                   columns=list('ABCD'))
df1

In [ ]:
df1.loc[['a', 'b', 'd'], :]

Accessing via label slices:

In [ ]:
df1.loc['d':, 'A':'C']

For getting a cross section using a label (equivalent to `df.xs('a')`):

In [ ]:
df1.loc['a']

For getting values with a boolean array:

In [ ]:
df1.loc['a'] > 0

In [ ]:
df1.loc[:, df1.loc['a'] > 0]

NA values in a boolean array propagate as `False`:

Changed in version 1.0.2.

In [ ]:
mask = pd.array([True, False, True, False, pd.NA, False], dtype="boolean")
mask

In [ ]:
df1[mask]

For getting a value explicitly:

In [ ]:
# this is also equivalent to ``df1.at['a','A']``
df1.loc['a', 'A']


<a id='indexing-slicing-with-labels'></a>

### Slicing with labels

When using `.loc` with slices, if both the start and the stop labels are
present in the index, then elements *located* between the two (including them)
are returned:

In [ ]:
s = pd.Series(list('abcde'), index=[0, 3, 2, 5, 4])
s.loc[3:5]

If at least one of the two is absent, but the index is sorted, and can be
compared against start and stop labels, then slicing will still work as
expected, by selecting labels which *rank* between the two:

In [ ]:
s.sort_index()

In [ ]:
s.sort_index().loc[1:6]

However, if at least one of the two is absent *and* the index is not sorted, an
error will be raised (since doing otherwise would be computationally expensive,
as well as potentially ambiguous for mixed type indexes). For instance, in the
above example, `s.loc[1:6]` would raise `KeyError`.

For the rationale behind this behavior, see
Endpoints are inclusive.

In [ ]:
s = pd.Series(list('abcdef'), index=[0, 3, 2, 5, 4, 2])
s.loc[3:5]

Also, if the index has duplicate labels *and* either the start or the stop label is duplicated,
an error will be raised. For instance, in the above example, `s.loc[2:5]` would raise a `KeyError`.

For more information about duplicate labels, see
Duplicate Labels.


<a id='indexing-integer'></a>

## Selection by position

Whether a copy or a reference is returned for a setting operation, may depend on the context.
This is sometimes called `chained assignment` and should be avoided.
See Returning a View versus Copy.

pandas provides a suite of methods in order to get **purely integer based indexing**. The semantics follow closely Python and NumPy slicing. These are `0-based` indexing. When slicing, the start bound is *included*, while the upper bound is *excluded*. Trying to use a non-integer, even a **valid** label will raise an `IndexError`.

The `.iloc` attribute is the primary access method. The following are valid inputs:

- An integer e.g. `5`.  
- A list or array of integers `[4, 3, 0]`.  
- A slice object with ints `1:7`.  
- A boolean array.  
- A `callable`, see Selection By Callable.  

In [ ]:
s1 = pd.Series(np.random.randn(5), index=list(range(0, 10, 2)))
s1

In [ ]:
s1.iloc[:3]

In [ ]:
s1.iloc[3]

Note that setting works as well:

In [ ]:
s1.iloc[:3] = 0
s1

With a DataFrame:

In [ ]:
df1 = pd.DataFrame(np.random.randn(6, 4),
                   index=list(range(0, 12, 2)),
                   columns=list(range(0, 8, 2)))
df1

Select via integer slicing:

In [ ]:
df1.iloc[:3]

In [ ]:
df1.iloc[1:5, 2:4]

Select via integer list:

In [ ]:
df1.iloc[[1, 3, 5], [1, 3]]

In [ ]:
df1.iloc[1:3, :]

In [ ]:
df1.iloc[:, 1:3]

In [ ]:
# this is also equivalent to ``df1.iat[1,1]``
df1.iloc[1, 1]

For getting a cross section using an integer position (equiv to `df.xs(1)`):

In [ ]:
df1.iloc[1]

Out of range slice indexes are handled gracefully just as in Python/NumPy.

In [ ]:
# these are allowed in Python/NumPy.
x = list('abcdef')
x

In [ ]:
x[4:10]

In [ ]:
x[8:10]

In [ ]:
s = pd.Series(x)
s

In [ ]:
s.iloc[4:10]

In [ ]:
s.iloc[8:10]

Note that using slices that go out of bounds can result in
an empty axis (e.g. an empty DataFrame being returned).

In [ ]:
dfl = pd.DataFrame(np.random.randn(5, 2), columns=list('AB'))
dfl

In [ ]:
dfl.iloc[:, 2:3]

In [ ]:
dfl.iloc[:, 1:3]

In [ ]:
dfl.iloc[4:6]

A single indexer that is out of bounds will raise an `IndexError`.
A list of indexers where any element is out of bounds will raise an
`IndexError`.

```python
>>> dfl.iloc[[4, 5, 6]]
IndexError: positional indexers are out-of-bounds

>>> dfl.iloc[:, 4]
IndexError: single positional indexer is out-of-bounds
```



<a id='indexing-callable'></a>

## Selection by callable

`.loc`, `.iloc`, and also `[]` indexing can accept a `callable` as indexer.
The `callable` must be a function with one argument (the calling Series or DataFrame) that returns valid output for indexing.

In [ ]:
df1 = pd.DataFrame(np.random.randn(6, 4),
                   index=list('abcdef'),
                   columns=list('ABCD'))
df1

In [ ]:
df1.loc[lambda df: df['A'] > 0, :]

In [ ]:
df1.loc[:, lambda df: ['A', 'B']]

In [ ]:
df1.iloc[:, lambda df: [0, 1]]

In [ ]:
df1[lambda df: df.columns[0]]

You can use callable indexing in `Series`.

In [ ]:
df1['A'].loc[lambda s: s > 0]

Using these methods / indexers, you can chain data selection operations
without using a temporary variable.

In [ ]:
bb = pd.read_csv('../data/baseball.csv', index_col='id')
(bb.groupby(['year', 'team']).sum()
   .loc[lambda df: df['r'] > 100])


<a id='combining-positional-and-label-based-indexing'></a>

## Combining positional and label-based indexing

If you wish to get the 0th and the 2nd elements from the index in the ‘A’ column, you can do:

In [ ]:
dfd = pd.DataFrame({'A': [1, 2, 3],
                    'B': [4, 5, 6]},
                   index=list('abc'))
dfd

In [ ]:
dfd.loc[dfd.index[[0, 2]], 'A']

This can also be expressed using `.iloc`, by explicitly getting locations on the indexers, and using
*positional* indexing to select things.

In [ ]:
dfd.iloc[[0, 2], dfd.columns.get_loc('A')]

For getting *multiple* indexers, using `.get_indexer`:

In [ ]:
dfd.iloc[[0, 2], dfd.columns.get_indexer(['A', 'B'])]


<a id='deprecate-loc-reindex-listlike'></a>

<a id='indexing-deprecate-loc-reindex-listlike'></a>

## Indexing with list with missing labels is deprecated

Changed in version 1.0.0.

Using `.loc` or `[]` with a list with one or more missing labels will no longer reindex, in favor of `.reindex`.

In prior versions, using `.loc[list-of-labels]` would work as long as *at least 1* of the keys was found (otherwise it
would raise a `KeyError`). This behavior was changed and will now raise a `KeyError` if at least one label is missing.
The recommended alternative is to use `.reindex()`.

For example.

In [ ]:
s = pd.Series([1, 2, 3])
s

Selection with all keys found is unchanged.

In [ ]:
s.loc[[1, 2]]

Previous behavior

```python
In [4]: s.loc[[1, 2, 3]]
Out[4]:
1    2.0
2    3.0
3    NaN
dtype: float64
```


Current behavior

```python
In [4]: s.loc[[1, 2, 3]]
Passing list-likes to .loc with any non-matching elements will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike

Out[4]:
1    2.0
2    3.0
3    NaN
dtype: float64
```


### Reindexing

The idiomatic way to achieve selecting potentially not-found elements is via `.reindex()`. See also the section on reindexing.

In [ ]:
s.reindex([1, 2, 3])

Alternatively, if you want to select only *valid* keys, the following is idiomatic and efficient; it is guaranteed to preserve the dtype of the selection.

In [ ]:
labels = [1, 2, 3]
s.loc[s.index.intersection(labels)]

Having a duplicated index will raise for a `.reindex()`:

In [ ]:
s = pd.Series(np.arange(4), index=['a', 'a', 'b', 'c'])
labels = ['c', 'd']

```python
In [17]: s.reindex(labels)
ValueError: cannot reindex on an axis with duplicate labels
```


Generally, you can intersect the desired labels with the current
axis, and then reindex.

In [ ]:
s.loc[s.index.intersection(labels)].reindex(labels)

However, this would *still* raise if your resulting index is duplicated.

```python
In [41]: labels = ['a', 'd']

In [42]: s.loc[s.index.intersection(labels)].reindex(labels)
ValueError: cannot reindex on an axis with duplicate labels
```



<a id='indexing-basics-partial-setting'></a>

## Selecting random samples

A random selection of rows or columns from a Series or DataFrame with the `sample()` method. The method will sample rows by default, and accepts a specific number of rows/columns to return, or a fraction of rows.

In [ ]:
s = pd.Series([0, 1, 2, 3, 4, 5])

# When no arguments are passed, returns 1 row.
s.sample()

In [ ]:
# One may specify either a number of rows:
s.sample(n=3)

In [ ]:
# Or a fraction of the rows:
s.sample(frac=0.5)

By default, `sample` will return each row at most once, but one can also sample with replacement
using the `replace` option:

In [ ]:
s = pd.Series([0, 1, 2, 3, 4, 5])

# Without replacement (default):
s.sample(n=6, replace=False)

In [ ]:
# With replacement:
s.sample(n=6, replace=True)

By default, each row has an equal probability of being selected, but if you want rows
to have different probabilities, you can pass the `sample` function sampling weights as
`weights`. These weights can be a list, a NumPy array, or a Series, but they must be of the same length as the object you are sampling. Missing values will be treated as a weight of zero, and inf values are not allowed. If weights do not sum to 1, they will be re-normalized by dividing all weights by the sum of the weights. For example:

In [ ]:
s = pd.Series([0, 1, 2, 3, 4, 5])
example_weights = [0, 0, 0.2, 0.2, 0.2, 0.4]
s.sample(n=3, weights=example_weights)

# Weights will be re-normalized automatically
example_weights2 = [0.5, 0, 0, 0, 0, 0]
s.sample(n=1, weights=example_weights2)

When applied to a DataFrame, you can use a column of the DataFrame as sampling weights
(provided you are sampling rows and not columns) by simply passing the name of the column
as a string.

In [ ]:
df2 = pd.DataFrame({'col1': [9, 8, 7, 6],
                    'weight_column': [0.5, 0.4, 0.1, 0]})
df2.sample(n=3, weights='weight_column')

`sample` also allows users to sample columns instead of rows using the `axis` argument.

In [ ]:
df3 = pd.DataFrame({'col1': [1, 2, 3], 'col2': [2, 3, 4]})
df3.sample(n=1, axis=1)

Finally, one can also set a seed for `sample`’s random number generator using the `random_state` argument, which will accept either an integer (as a seed) or a NumPy RandomState object.

In [ ]:
df4 = pd.DataFrame({'col1': [1, 2, 3], 'col2': [2, 3, 4]})

# With a given seed, the sample will always draw the same rows.
df4.sample(n=2, random_state=2)

In [ ]:
df4.sample(n=2, random_state=2)

## Setting with enlargement

The `.loc/[]` operations can perform enlargement when setting a non-existent key for that axis.

In the `Series` case this is effectively an appending operation.

In [ ]:
se = pd.Series([1, 2, 3])
se

In [ ]:
se[5] = 5.
se

A `DataFrame` can be enlarged on either axis via `.loc`.

In [ ]:
dfi = pd.DataFrame(np.arange(6).reshape(3, 2),
                   columns=['A', 'B'])
dfi

In [ ]:
dfi.loc[:, 'C'] = dfi.loc[:, 'A']
dfi

This is like an `append` operation on the `DataFrame`.

In [ ]:
dfi.loc[3] = 5
dfi


<a id='indexing-basics-get-value'></a>

## Fast scalar value getting and setting

Since indexing with `[]` must handle a lot of cases (single-label access,
slicing, boolean indexing, etc.), it has a bit of overhead in order to figure
out what you’re asking for. If you only want to access a scalar value, the
fastest way is to use the `at` and `iat` methods, which are implemented on
all of the data structures.

Similarly to `loc`, `at` provides **label** based scalar lookups, while, `iat` provides **integer** based lookups analogously to `iloc`

In [ ]:
s.iat[5]

In [ ]:
df.at[dates[5], 'A']

In [ ]:
df.iat[3, 0]

You can also set using these same indexers.

In [ ]:
df.at[dates[5], 'E'] = 7

In [ ]:
df.iat[3, 0] = 7

`at` may enlarge the object in-place as above if the indexer is missing.

In [ ]:
df.at[dates[-1] + pd.Timedelta('1 day'), 0] = 7

In [ ]:
df

## Boolean indexing


<a id='indexing-boolean'></a>
Another common operation is the use of boolean vectors to filter the data.
The operators are: `|` for `or`, `&` for `and`, and `~` for `not`.
These **must** be grouped by using parentheses, since by default Python will
evaluate an expression such as `df['A'] > 2 & df['B'] < 3` as
`df['A'] > (2 & df['B']) < 3`, while the desired evaluation order is
`(df['A'] > 2) & (df['B'] < 3)`.

Using a boolean vector to index a Series works exactly as in a NumPy ndarray:

In [ ]:
s = pd.Series(range(-3, 4))
s

In [ ]:
s[s > 0]

In [ ]:
s[(s < -1) | (s > 0.5)]

In [ ]:
s[~(s < 0)]

You may select rows from a DataFrame using a boolean vector the same length as
the DataFrame’s index (for example, something derived from one of the columns
of the DataFrame):

In [ ]:
df[df['A'] > 0]

List comprehensions and the `map` method of Series can also be used to produce
more complex criteria:

In [ ]:
df2 = pd.DataFrame({'a': ['one', 'one', 'two', 'three', 'two', 'one', 'six'],
                    'b': ['x', 'y', 'y', 'x', 'y', 'x', 'x'],
                    'c': np.random.randn(7)})

# only want 'two' or 'three'
criterion = df2['a'].map(lambda x: x.startswith('t'))
df2[criterion]

In [ ]:
# equivalent but slower
df2[[x.startswith('t') for x in df2['a']]]

In [ ]:
# Multiple criteria
df2[criterion & (df2['b'] == 'x')]

With the choice methods Selection by Label, Selection by Position,
and Advanced Indexing you may select along more than one axis using boolean vectors combined with other indexing expressions.

In [ ]:
df2.loc[criterion & (df2['b'] == 'x'), 'b':'c']

`iloc` supports two kinds of boolean indexing. If the indexer is a boolean `Series`,
an error will be raised. For instance, in the following example, `df.iloc[s.values, 1]` is ok.
The boolean indexer is an array. But `df.iloc[s, 1]` would raise `ValueError`.

In [ ]:
df = pd.DataFrame([[1, 2], [3, 4], [5, 6]],
                  index=list('abc'),
                  columns=['A', 'B'])
s = (df['A'] > 2)
s

In [ ]:
df.loc[s, 'B']

In [ ]:
df.iloc[s.values, 1]


<a id='indexing-basics-indexing-isin'></a>

## Indexing with isin

Consider the `isin()` method of `Series`, which returns a boolean
vector that is true wherever the `Series` elements exist in the passed list.
This allows you to select rows where one or more columns have values you want:

In [ ]:
s = pd.Series(np.arange(5), index=np.arange(5)[::-1], dtype='int64')
s

In [ ]:
s.isin([2, 4, 6])

In [ ]:
s[s.isin([2, 4, 6])]

The same method is available for `Index` objects and is useful for the cases
when you don’t know which of the sought labels are in fact present:

In [ ]:
s[s.index.isin([2, 4, 6])]

In [ ]:
# compare it to the following
s.reindex([2, 4, 6])

In addition to that, `MultiIndex` allows selecting a separate level to use
in the membership check:

In [ ]:
s_mi = pd.Series(np.arange(6),
                 index=pd.MultiIndex.from_product([[0, 1], ['a', 'b', 'c']]))
s_mi
s_mi.iloc[s_mi.index.isin([(1, 'a'), (2, 'b'), (0, 'c')])]
s_mi.iloc[s_mi.index.isin(['a', 'c', 'e'], level=1)]

DataFrame also has an `isin()` method.  When calling `isin`, pass a set of
values as either an array or dict.  If values is an array, `isin` returns
a DataFrame of booleans that is the same shape as the original DataFrame, with True
wherever the element is in the sequence of values.

In [ ]:
df = pd.DataFrame({'vals': [1, 2, 3, 4], 'ids': ['a', 'b', 'f', 'n'],
                   'ids2': ['a', 'n', 'c', 'n']})
values = ['a', 'b', 1, 3]
df.isin(values)

Oftentimes you’ll want to match certain values with certain columns.
Just make values a `dict` where the key is the column, and the value is
a list of items you want to check for.

In [ ]:
values = {'ids': ['a', 'b'], 'vals': [1, 3]}
df.isin(values)

Combine DataFrame’s `isin` with the `any()` and `all()` methods to
quickly select subsets of your data that meet a given criteria.
To select a row where each column meets its own criterion:

In [ ]:
values = {'ids': ['a', 'b'], 'ids2': ['a', 'c'], 'vals': [1, 3]}
row_mask = df.isin(values).all(1)
df[row_mask]


<a id='indexing-where-mask'></a>

## The `where()` Method and Masking

Selecting values from a Series with a boolean vector generally returns a
subset of the data. To guarantee that selection output has the same shape as
the original data, you can use the `where` method in `Series` and `DataFrame`.

To return only the selected rows:

In [ ]:
s[s > 0]

To return a Series of the same shape as the original:

In [ ]:
s.where(s > 0)

Selecting values from a DataFrame with a boolean criterion now also preserves
input data shape. `where` is used under the hood as the implementation.
The code below is equivalent to `df.where(df < 0)`.

In [ ]:
dates = pd.date_range('1/1/2000', periods=8)
df = pd.DataFrame(np.random.randn(8, 4),
                  index=dates, columns=['A', 'B', 'C', 'D'])

In [ ]:
df[df < 0]

In addition, `where` takes an optional `other` argument for replacement of
values where the condition is False, in the returned copy.

In [ ]:
df.where(df < 0, -df)

You may wish to set values based on some boolean criteria.
This can be done intuitively like so:

In [ ]:
s2 = s.copy()
s2[s2 < 0] = 0
s2

df2 = df.copy()
df2[df2 < 0] = 0
df2

By default, `where` returns a modified copy of the data. There is an
optional parameter `inplace` so that the original data can be modified
without creating a copy:

In [ ]:
df_orig = df.copy()
df_orig.where(df > 0, -df, inplace=True)
df_orig

>**Note**
>
>The signature for `DataFrame.where()` differs from `numpy.where()`.
Roughly `df1.where(m, df2)` is equivalent to `np.where(m, df1, df2)`.

In [ ]:
df.where(df < 0, -df) == np.where(df < 0, df, -df)

**Alignment**

Furthermore, `where` aligns the input boolean condition (ndarray or DataFrame),
such that partial selection with setting is possible. This is analogous to
partial setting via `.loc` (but on the contents rather than the axis labels).

In [ ]:
df2 = df.copy()
df2[df2[1:4] > 0] = 3
df2

Where can also accept `axis` and `level` parameters to align the input when
performing the `where`.

In [ ]:
df2 = df.copy()
df2.where(df2 > 0, df2['A'], axis='index')

This is equivalent to (but faster than) the following.

In [ ]:
df2 = df.copy()
df.apply(lambda x, y: x.where(x > 0, y), y=df['A'])

`where` can accept a callable as condition and `other` arguments. The function must
be with one argument (the calling Series or DataFrame) and that returns valid output
as condition and `other` argument.

In [ ]:
df3 = pd.DataFrame({'A': [1, 2, 3],
                    'B': [4, 5, 6],
                    'C': [7, 8, 9]})
df3.where(lambda x: x > 4, lambda x: x + 10)

### Mask

`mask()` is the inverse boolean operation of `where`.

In [ ]:
s.mask(s >= 0)

In [ ]:
df.mask(df >= 0)


<a id='indexing-np-where'></a>

## Setting with enlargement conditionally using `numpy()`

An alternative to `where()` is to use `numpy.where()`.
Combined with setting a new column, you can use it to enlarge a DataFrame where the
values are determined conditionally.

Consider you have two choices to choose from in the following DataFrame. And you want to
set a new column color to ‘green’ when the second column has ‘Z’.  You can do the
following:

In [ ]:
df = pd.DataFrame({'col1': list('ABBC'), 'col2': list('ZZXY')})
df['color'] = np.where(df['col2'] == 'Z', 'green', 'red')
df

If you have multiple conditions, you can use `numpy.select()` to achieve that.  Say
corresponding to three conditions there are three choice of colors, with a fourth color
as a fallback, you can do the following.

In [ ]:
conditions = [
    (df['col2'] == 'Z') & (df['col1'] == 'A'),
    (df['col2'] == 'Z') & (df['col1'] == 'B'),
    (df['col1'] == 'B')
]
choices = ['yellow', 'blue', 'purple']
df['color'] = np.select(conditions, choices, default='black')
df


<a id='indexing-query'></a>

## The `query()` Method

`DataFrame` objects have a `query()`
method that allows selection using an expression.

You can get the value of the frame where column `b` has values
between the values of columns `a` and `c`. For example:

In [ ]:
n = 10
df = pd.DataFrame(np.random.rand(n, 3), columns=list('abc'))
df

In [ ]:
# pure python
df[(df['a'] < df['b']) & (df['b'] < df['c'])]

In [ ]:
# query
df.query('(a < b) & (b < c)')

Do the same thing but fall back on a named index if there is no column
with the name `a`.

In [ ]:
df = pd.DataFrame(np.random.randint(n / 2, size=(n, 2)), columns=list('bc'))
df.index.name = 'a'
df

In [ ]:
df.query('a < b and b < c')

If instead you don’t want to or cannot name your index, you can use the name
`index` in your query expression:

In [ ]:
df = pd.DataFrame(np.random.randint(n, size=(n, 2)), columns=list('bc'))
df

In [ ]:
df.query('index < b < c')

>**Note**
>
>If the name of your index overlaps with a column name, the column name is
given precedence. For example,

In [ ]:
df = pd.DataFrame({'a': np.random.randint(5, size=5)})
df.index.name = 'a'
df.query('a > 2')  # uses the column 'a', not the index

You can still use the index in a query expression by using the special
identifier ‘index’:

In [ ]:
df.query('index > 2')

If for some reason you have a column named `index`, then you can refer to
the index as `ilevel_0` as well, but at this point you should consider
renaming your columns to something less ambiguous.

### `MultiIndex` `query()` Syntax

You can also use the levels of a `DataFrame` with a
`MultiIndex` as if they were columns in the frame:

In [ ]:
n = 10
colors = np.random.choice(['red', 'green'], size=n)
foods = np.random.choice(['eggs', 'ham'], size=n)
colors

In [ ]:
foods

In [ ]:
index = pd.MultiIndex.from_arrays([colors, foods], names=['color', 'food'])
df = pd.DataFrame(np.random.randn(n, 2), index=index)
df

In [ ]:
df.query('color == "red"')

If the levels of the `MultiIndex` are unnamed, you can refer to them using
special names:

In [ ]:
df.index.names = [None, None]
df

In [ ]:
df.query('ilevel_0 == "red"')

The convention is `ilevel_0`, which means “index level 0” for the 0th level
of the `index`.

### `query()` Use Cases

A use case for `query()` is when you have a collection of
`DataFrame` objects that have a subset of column names (or index
levels/names) in common. You can pass the same query to both frames *without*
having to specify which frame you’re interested in querying

In [ ]:
df = pd.DataFrame(np.random.rand(n, 3), columns=list('abc'))
df

In [ ]:
df2 = pd.DataFrame(np.random.rand(n + 2, 3), columns=df.columns)
df2

In [ ]:
expr = '0.0 <= a <= c <= 0.5'
map(lambda frame: frame.query(expr), [df, df2])

### `query()` Python versus pandas Syntax Comparison

Full numpy-like syntax:

In [ ]:
df = pd.DataFrame(np.random.randint(n, size=(n, 3)), columns=list('abc'))
df

In [ ]:
df.query('(a < b) & (b < c)')
df[(df['a'] < df['b']) & (df['b'] < df['c'])]

Slightly nicer by removing the parentheses (comparison operators bind tighter
than `&` and `|`):

In [ ]:
df.query('a < b & b < c')

Use English instead of symbols:

In [ ]:
df.query('a < b and b < c')

Pretty close to how you might write it on paper:

In [ ]:
df.query('a < b < c')

### The `in` and `not in` operators

`query()` also supports special use of Python’s `in` and
`not in` comparison operators, providing a succinct syntax for calling the
`isin` method of a `Series` or `DataFrame`.

In [ ]:
# get all rows where columns "a" and "b" have overlapping values
df = pd.DataFrame({'a': list('aabbccddeeff'), 'b': list('aaaabbbbcccc'),
                   'c': np.random.randint(5, size=12),
                   'd': np.random.randint(9, size=12)})
df

In [ ]:
df.query('a in b')

# How you'd do it in pure Python
df[df['a'].isin(df['b'])]

In [ ]:
df.query('a not in b')

In [ ]:
# pure Python
df[~df['a'].isin(df['b'])]

You can combine this with other expressions for very succinct queries:

In [ ]:
# rows where cols a and b have overlapping values
# and col c's values are less than col d's
df.query('a in b and c < d')

In [ ]:
# pure Python
df[df['b'].isin(df['a']) & (df['c'] < df['d'])]

>**Note**
>
>Note that `in` and `not in` are evaluated in Python, since `numexpr`
has no equivalent of this operation. However, **only the** `in`/`not in`
**expression itself** is evaluated in vanilla Python. For example, in the
expression
>
>    df.query('a in b + c + d')
>
>``(b + c + d)`` is evaluated by ``numexpr`` and *then* the ``in``
operation is evaluated in plain Python. In general, any operations that can
be evaluated using ``numexpr`` will be. 

### Special use of the `==` operator with `list` objects

Comparing a `list` of values to a column using `==`/`!=` works similarly
to `in`/`not in`.

In [ ]:
df.query('b == ["a", "b", "c"]')

In [ ]:
# pure Python
df[df['b'].isin(["a", "b", "c"])]

In [ ]:
df.query('c == [1, 2]')

In [ ]:
df.query('c != [1, 2]')

In [ ]:
# using in/not in
df.query('[1, 2] in c')

In [ ]:
df.query('[1, 2] not in c')

In [ ]:
# pure Python
df[df['c'].isin([1, 2])]

### Boolean operators

You can negate boolean expressions with the word `not` or the `~` operator.

In [ ]:
df = pd.DataFrame(np.random.rand(n, 3), columns=list('abc'))
df['bools'] = np.random.rand(len(df)) > 0.5
df.query('~bools')

In [ ]:
df.query('not bools')

In [ ]:
df.query('not bools') == df[~df['bools']]

Of course, expressions can be arbitrarily complex too:

In [ ]:
# short query syntax
shorter = df.query('a < b < c and (not bools) or bools > 2')

# equivalent in pure Python
longer = df[(df['a'] < df['b'])
            & (df['b'] < df['c'])
            & (~df['bools'])
            | (df['bools'] > 2)]

shorter

In [ ]:
longer

In [ ]:
shorter == longer

### Performance of `query()`

`DataFrame.query()` using `numexpr` is slightly faster than Python for
large frames.

<img src="../_static/query-perf.png" style="">

>**Note**
>
>You will only see the performance benefits of using the `numexpr` engine
with `DataFrame.query()` if your frame has more than approximately 200,000
rows.

> <img src="../_static/query-perf-small.png" style="">

This plot was created using a `DataFrame` with 3 columns each containing
floating point values generated using `numpy.random.randn()`.

In [ ]:
df = pd.DataFrame(np.random.randn(8, 4),
                  index=dates, columns=['A', 'B', 'C', 'D'])
df2 = df.copy()

## Duplicate data


<a id='indexing-duplicate'></a>
If you want to identify and remove duplicate rows in a DataFrame,  there are
two methods that will help: `duplicated` and `drop_duplicates`. Each
takes as an argument the columns to use to identify duplicated rows.

- `duplicated` returns a boolean vector whose length is the number of rows, and which indicates whether a row is duplicated.  
- `drop_duplicates` removes duplicate rows.  


By default, the first observed row of a duplicate set is considered unique, but
each method has a `keep` parameter to specify targets to be kept.

- `keep='first'` (default): mark / drop duplicates except for the first occurrence.  
- `keep='last'`: mark / drop duplicates except for the last occurrence.  
- `keep=False`: mark  / drop all duplicates.  

In [ ]:
df2 = pd.DataFrame({'a': ['one', 'one', 'two', 'two', 'two', 'three', 'four'],
                    'b': ['x', 'y', 'x', 'y', 'x', 'x', 'x'],
                    'c': np.random.randn(7)})
df2

In [ ]:
df2.duplicated('a')

In [ ]:
df2.duplicated('a', keep='last')

In [ ]:
df2.duplicated('a', keep=False)

In [ ]:
df2.drop_duplicates('a')

In [ ]:
df2.drop_duplicates('a', keep='last')

In [ ]:
df2.drop_duplicates('a', keep=False)

Also, you can pass a list of columns to identify duplications.

In [ ]:
df2.duplicated(['a', 'b'])

In [ ]:
df2.drop_duplicates(['a', 'b'])

To drop duplicates by index value, use `Index.duplicated` then perform slicing.
The same set of options are available for the `keep` parameter.

In [ ]:
df3 = pd.DataFrame({'a': np.arange(6),
                    'b': np.random.randn(6)},
                   index=['a', 'a', 'b', 'c', 'b', 'a'])
df3

In [ ]:
df3.index.duplicated()

In [ ]:
df3[~df3.index.duplicated()]

In [ ]:
df3[~df3.index.duplicated(keep='last')]

In [ ]:
df3[~df3.index.duplicated(keep=False)]


<a id='indexing-dictionarylike'></a>

## Dictionary-like `get()` method

Each of Series or DataFrame have a `get` method which can return a
default value.

In [ ]:
s = pd.Series([1, 2, 3], index=['a', 'b', 'c'])
s.get('a')  # equivalent to s['a']

In [ ]:
s.get('x', default=-1)


<a id='indexing-lookup'></a>

## Looking up values by index/column labels

Sometimes you want to extract a set of values given a sequence of row labels
and column labels, this can be achieved by `pandas.factorize`  and NumPy indexing.
For instance:

In [ ]:
df = pd.DataFrame({'col': ["A", "A", "B", "B"],
                   'A': [80, 23, np.nan, 22],
                   'B': [80, 55, 76, 67]})
df

In [ ]:
idx, cols = pd.factorize(df['col'])
df.reindex(cols, axis=1).to_numpy()[np.arange(len(df)), idx]

Formerly this could be achieved with the dedicated `DataFrame.lookup` method
which was deprecated in version 1.2.0.


<a id='indexing-class'></a>

## Index objects

The pandas `Index` class and its subclasses can be viewed as
implementing an *ordered multiset*. Duplicates are allowed. However, if you try
to convert an `Index` object with duplicate entries into a
`set`, an exception will be raised.

`Index` also provides the infrastructure necessary for
lookups, data alignment, and reindexing. The easiest way to create an
`Index` directly is to pass a `list` or other sequence to
`Index`:

In [ ]:
index = pd.Index(['e', 'd', 'a', 'b'])
index

In [ ]:
'd' in index

You can also pass a `name` to be stored in the index:

In [ ]:
index = pd.Index(['e', 'd', 'a', 'b'], name='something')
index.name

The name, if set, will be shown in the console display:

In [ ]:
index = pd.Index(list(range(5)), name='rows')
columns = pd.Index(['A', 'B', 'C'], name='cols')
df = pd.DataFrame(np.random.randn(5, 3), index=index, columns=columns)
df

In [ ]:
df['A']


<a id='indexing-set-metadata'></a>

### Setting metadata

Indexes are “mostly immutable”, but it is possible to set and change their
`name` attribute. You can use the `rename`, `set_names` to set these attributes
directly, and they default to returning a copy.

See Advanced Indexing for usage of MultiIndexes.

In [ ]:
ind = pd.Index([1, 2, 3])
ind.rename("apple")
ind

In [ ]:
ind.set_names(["apple"], inplace=True)
ind.name = "bob"
ind

`set_names`, `set_levels`, and `set_codes` also take an optional
`level` argument

In [ ]:
index = pd.MultiIndex.from_product([range(3), ['one', 'two']], names=['first', 'second'])
index

In [ ]:
index.levels[1]

In [ ]:
index.set_levels(["a", "b"], level=1)


<a id='indexing-set-ops'></a>

### Set operations on Index objects

The two main operations are `union` and `intersection`.
Difference is provided via the `.difference()` method.

In [ ]:
a = pd.Index(['c', 'b', 'a'])
b = pd.Index(['c', 'e', 'd'])
a.difference(b)

Also available is the `symmetric_difference` operation, which returns elements
that appear in either `idx1` or `idx2`, but not in both. This is
equivalent to the Index created by `idx1.difference(idx2).union(idx2.difference(idx1))`,
with duplicates dropped.

In [ ]:
idx1 = pd.Index([1, 2, 3, 4])
idx2 = pd.Index([2, 3, 4, 5])
idx1.symmetric_difference(idx2)

>**Note**
>
>The resulting index from a set operation will be sorted in ascending order.

When performing `Index.union()` between indexes with different dtypes, the indexes
must be cast to a common dtype. Typically, though not always, this is object dtype. The
exception is when performing a union between integer and float data. In this case, the
integer values are converted to float

In [ ]:
idx1 = pd.Index([0, 1, 2])
idx2 = pd.Index([0.5, 1.5])
idx1.union(idx2)


<a id='indexing-missing'></a>

### Missing values

Even though `Index` can hold missing values (`NaN`), it should be avoided
if you do not want any unexpected results. For example, some operations
exclude missing values implicitly.

`Index.fillna` fills missing values with specified scalar value.

In [ ]:
idx1 = pd.Index([1, np.nan, 3, 4])
idx1

In [ ]:
idx1.fillna(2)

idx2 = pd.DatetimeIndex([pd.Timestamp('2011-01-01'),
                         pd.NaT,
                         pd.Timestamp('2011-01-03')])
idx2

In [ ]:
idx2.fillna(pd.Timestamp('2011-01-02'))

## Set / reset index

Occasionally you will load or create a data set into a DataFrame and want to
add an index after you’ve already done so. There are a couple of different
ways.


<a id='indexing-set-index'></a>

### Set an index

DataFrame has a `set_index()` method which takes a column name
(for a regular `Index`) or a list of column names (for a `MultiIndex`).
To create a new, re-indexed DataFrame:

In [ ]:
data = pd.DataFrame({'a': ['bar', 'bar', 'foo', 'foo'],
                     'b': ['one', 'two', 'one', 'two'],
                     'c': ['z', 'y', 'x', 'w'],
                     'd': [1., 2., 3, 4]})

In [ ]:
data

In [ ]:
indexed1 = data.set_index('c')
indexed1

In [ ]:
indexed2 = data.set_index(['a', 'b'])
indexed2

The `append` keyword option allow you to keep the existing index and append
the given columns to a MultiIndex:

In [ ]:
frame = data.set_index('c', drop=False)
frame = frame.set_index(['a', 'b'], append=True)
frame

Other options in `set_index` allow you not drop the index columns or to add
the index in-place (without creating a new object):

In [ ]:
data.set_index('c', drop=False)
data.set_index(['a', 'b'], inplace=True)
data

### Reset the index

As a convenience, there is a new function on DataFrame called
`reset_index()` which transfers the index values into the
DataFrame’s columns and sets a simple integer index.
This is the inverse operation of `set_index()`.

In [ ]:
data
data.reset_index()

The output is more similar to a SQL table or a record array. The names for the
columns derived from the index are the ones stored in the `names` attribute.

You can use the `level` keyword to remove only a portion of the index:

In [ ]:
frame
frame.reset_index(level=1)

`reset_index` takes an optional parameter `drop` which if true simply
discards the index, instead of putting index values in the DataFrame’s columns.

### Adding an ad hoc index

If you create an index yourself, you can just assign it to the `index` field:

```python
data.index = index
```



<a id='indexing-view-versus-copy'></a>

## Returning a view versus a copy

When setting values in a pandas object, care must be taken to avoid what is called
`chained indexing`. Here is an example.

In [ ]:
dfmi = pd.DataFrame([list('abcd'),
                     list('efgh'),
                     list('ijkl'),
                     list('mnop')],
                    columns=pd.MultiIndex.from_product([['one', 'two'],
                                                        ['first', 'second']]))
dfmi

Compare these two access methods:

In [ ]:
dfmi['one']['second']

In [ ]:
dfmi.loc[:, ('one', 'second')]

These both yield the same results, so which should you use? It is instructive to understand the order
of operations on these and why method 2 (`.loc`) is much preferred over method 1 (chained `[]`).

`dfmi['one']` selects the first level of the columns and returns a DataFrame that is singly-indexed.
Then another Python operation `dfmi_with_one['second']` selects the series indexed by `'second'`.
This is indicated by the variable `dfmi_with_one` because pandas sees these operations as separate events.
e.g. separate calls to `__getitem__`, so it has to treat them as linear operations, they happen one after another.

Contrast this to `df.loc[:,('one','second')]` which passes a nested tuple of `(slice(None),('one','second'))` to a single call to
`__getitem__`. This allows pandas to deal with this as a single entity. Furthermore this order of operations *can* be significantly
faster, and allows one to index *both* axes if so desired.

### Why does assignment fail when using chained indexing?

The problem in the previous section is just a performance issue. What’s up with
the `SettingWithCopy` warning? We don’t **usually** throw warnings around when
you do something that might cost a few extra milliseconds!

But it turns out that assigning to the product of chained indexing has
inherently unpredictable results. To see this, think about how the Python
interpreter executes this code:

In [ ]:
value = None

```python
dfmi.loc[:, ('one', 'second')] = value
# becomes
dfmi.loc.__setitem__((slice(None), ('one', 'second')), value)
```


But this code is handled differently:

```python
dfmi['one']['second'] = value
# becomes
dfmi.__getitem__('one').__setitem__('second', value)
```


See that `__getitem__` in there? Outside of simple cases, it’s very hard to
predict whether it will return a view or a copy (it depends on the memory layout
of the array, about which pandas makes no guarantees), and therefore whether
the `__setitem__` will modify `dfmi` or a temporary object that gets thrown
out immediately afterward. **That’s** what `SettingWithCopy` is warning you
about!

>**Note**
>
>You may be wondering whether we should be concerned about the `loc`
property in the first example. But `dfmi.loc` is guaranteed to be `dfmi`
itself with modified indexing behavior, so `dfmi.loc.__getitem__` /
`dfmi.loc.__setitem__` operate on `dfmi` directly. Of course,
`dfmi.loc.__getitem__(idx)` may be a view or a copy of `dfmi`.

Sometimes a `SettingWithCopy` warning will arise at times when there’s no
obvious chained indexing going on. **These** are the bugs that
`SettingWithCopy` is designed to catch! pandas is probably trying to warn you
that you’ve done this:

```python
def do_something(df):
    foo = df[['bar', 'baz']]  # Is foo a view? A copy? Nobody knows!
    # ... many lines here ...
    # We don't know whether this will modify df or not!
    foo['quux'] = value
    return foo
```


Yikes!


<a id='indexing-evaluation-order'></a>

### Evaluation order matters

When you use chained indexing, the order and type of the indexing operation
partially determine whether the result is a slice into the original object, or
a copy of the slice.

pandas has the `SettingWithCopyWarning` because assigning to a copy of a
slice is frequently not intentional, but a mistake caused by chained indexing
returning a copy where a slice was expected.

If you would like pandas to be more or less trusting about assignment to a
chained indexing expression, you can set the option
`mode.chained_assignment` to one of these values:

- `'warn'`, the default, means a `SettingWithCopyWarning` is printed.  
- `'raise'` means pandas will raise a `SettingWithCopyException`
  you have to deal with.  
- `None` will suppress the warnings entirely.  

In [ ]:
dfb = pd.DataFrame({'a': ['one', 'one', 'two',
                          'three', 'two', 'one', 'six'],
                    'c': np.arange(7)})

# This will show the SettingWithCopyWarning but the frame values will be set
dfb['c'][dfb['a'].str.startswith('o')] = 42

This however is operating on a copy and will not work.

In [ ]:
pd.set_option('mode.chained_assignment','warn')
dfb[dfb['a'].str.startswith('o')]['c'] = 42
'''
SettingWithCopyWarning:
     A value is trying to be set on a copy of a slice from a DataFrame.
     Try using .loc[row_index,col_indexer] = value instead
'''


A chained assignment can also crop up in setting in a mixed dtype frame.

>**Note**
>
>These setting rules apply to all of `.loc/.iloc`.

The following is the recommended access method using `.loc` for multiple items (using `mask`) and a single item using a fixed index:

In [ ]:
dfc = pd.DataFrame({'a': ['one', 'one', 'two',
                          'three', 'two', 'one', 'six'],
                    'c': np.arange(7)})
dfd = dfc.copy()
# Setting multiple items using a mask
mask = dfd['a'].str.startswith('o')
dfd.loc[mask, 'c'] = 42
dfd

In [ ]:
# Setting a single item
dfd = dfc.copy()
dfd.loc[2, 'c'] = 11
dfd

The following *can* work at times, but it is not guaranteed to, and therefore should be avoided:

In [ ]:
dfd = dfc.copy()
dfd['a'][2] = 111
dfd

Last, the subsequent example will **not** work at all, and so should be avoided:

In [ ]:
pd.set_option('mode.chained_assignment','raise')
dfd.loc[0]['a'] = 1111
#SettingWithCopyError: 
#A value is trying to be set on a copy of a slice from a DataFrame
#See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


The chained assignment warnings / exceptions are aiming to inform the user of a possibly invalid
assignment. There may be false positives; situations where a chained assignment is inadvertently
reported.